In [27]:
import cantera as ct 
import cantera.ck2cti
from rmgpy import chemkin
from rmgpy import kinetics
import numpy as np
import subprocess
import csv
import copy
import os

Function: Load_models

In [28]:
def load_models(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2):
    
    full_path_1 = full_path_1
    inp_path_1 = full_path_1 + chemkin_1
    dict_path_1 = full_path_1 + dictionary_1
    model_1_species, model_1_reactions = chemkin.load_chemkin_file(inp_path_1, dict_path_1)
    
    full_path_2 = full_path_2
    inp_path_2 = full_path_2 + chemkin_2
    dict_path_2 = full_path_2 + dictionary_2
    model_2_species, model_2_reactions = chemkin.load_chemkin_file(inp_path_2, dict_path_2)

    different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq__2, model_1_species, model_1_reactions, model_2_species, model_2_reactions  = execute(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2) 
    return different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq__2, model_1_species, model_1_reactions, model_2_species, model_2_reactions 


Functions: Compare Reactions

In [61]:
!pwd

/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/NK_120_to_140


In [66]:
#therefore, D = 1 = 130, N = 2 = 120
def compare_rxn_equations(model_1_reactions, model_2_reactions):
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (model_1_reactions, model_2_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    for reactionList, reaction in to_remove:
        reactionList.remove(reaction)
    print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    unique_reactions_1 = []
    unique_reactions_2 = []
    copy_of_model_2_reactions = model_2_reactions.copy() 
    copy_of_model_1_reactions = model_1_reactions.copy() 

    
    for rxn_2 in copy_of_model_2_reactions:
        for rxn_1 in copy_of_model_1_reactions:  # make a copy so you don't remove from the list you are iterating over
            if rxn_2.is_isomorphic(rxn_1):
                common_reactions.append([rxn_2, rxn_1])
                #common_reactions_equations.append([str(rxnN), str(rxnD)])
                # Remove reaction 2 from being chosen a second time.
                # Let each reaction only appear only once in the diff comparison.
                # Otherwise this miscounts number of reactions in model 2.
                copy_of_model_1_reactions.remove(rxn_1)
 
    
    for rxn_2 in copy_of_model_2_reactions:
        for r_2, r_1 in common_reactions:
            if rxn_2.is_isomorphic(r_2):
                if rxn_2 is not r_2:
                    print(str(rxn_2))
                break
        else:
            unique_reactions_2.append(rxn_2)
    for rxn_1 in copy_of_model_1_reactions:
        for r_2, r_1 in common_reactions:
            if rxn_1.is_isomorphic(r_1):
                break
        else:
            unique_reactions_1.append(rxn_1)
            
    different_kinetics_reactions = []
            
    for r_2, r_1 in common_reactions: 
        if (r_2.kinetics.is_identical_to(r_1.kinetics)==False) and (r_2.kinetics.is_similar_to(r_1.kinetics)==False):
            different_kinetics_reactions.append(r_1)
            
    return different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2
    
    
    

def details(different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2):
    
    '''
    Takes rmg objects and extracts readable info (equations, indexes, etc)
    '''
    
    diff_kin_equations = [str(i) for i in different_kinetics_reactions]
    comm_equations = [str(i) for i in common_reactions]
    
    uniq_rxns_eq_1 = [str(i) for i in unique_reactions_1]

    uniq_rxns_eq_2 = [str(i) for i in unique_reactions_2]
    
    return diff_kin_equations, comm_equations, uniq_rxns_eq_1, uniq_rxns_eq_2 
    
    
    
    
    
    
    
def execute(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2): 
    '''
    loads in chemkin files and runs commands to execute the file
    '''
    full_path_2 = full_path_2
    inp_path_2 = full_path_2 + chemkin_2
    dict_path_2 = full_path_2 + dictionary_2
    model_2_species, model_2_reactions = chemkin.load_chemkin_file(inp_path_2, dict_path_2)

    full_path_1 = full_path_1
    inp_path_1 = full_path_1 + chemkin_1
    dict_path_1 = full_path_1 + dictionary_1
    model_1_species, model_1_reactions = chemkin.load_chemkin_file(inp_path_1, dict_path_1)
    
    different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2 = compare_rxn_equations(model_1_reactions, model_2_reactions)
    diff_kin_equations, comm_equations, uniq_rxns_eq_1, uniq_rxns_eq_2  = details(different_kinetics_reactions, common_reactions, unique_reactions_1, unique_reactions_2)
    return different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq_2, model_1_species, model_1_reactions, model_2_species, model_2_reactions 

    

Functions: One_Flip_120_140

In [4]:
def execute_flip(different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq__2, model_1_species, model_1_reactions, model_2_species, model_2_reactions)
    


    execute_flip()

SyntaxError: invalid syntax (133136088.py, line 1)

Run 

In [67]:
different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactions_1, uniq_rxns_eq_1, unique_reactions_2, uniq_rxns_eq_2, model_1_species, model_1_reactions, model_2_species, model_2_reactions  = load_models(
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/models/Nora/2_BTP_seed/chemkin/",
    "copies/copy_chem_130.inp",
    "species_dictionary.txt",
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/models/Nora/2_BTP_seed/chemkin/",
    "copies/copy_chem_129.inp",
    "species_dictionary.txt"
    )


#load_models(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2)
# execute(full_path_1, chemkin_1, dictionary_1, full_path_2, chemkin_2, dictionary_2) 

#therefore, D = 1 = 130, N = 2 = 120

Following reactions removed because they have unknown species: []
CH2CHO(35) <=> CO(15) + CH3(19)
CO(15) + CH3(19) <=> HCO(17) + CH2(T)(18)
O2(157) + 2-BTP(1) <=> O2(4) + 2-BTP(1)
CF3CCH(84) + 2-BTP(1) <=> CF3CCH(84) + S(164)
CF3CCH(84) + 2-BTP(1) <=> CF3CCH(84) + S(164)
CF3CCH(84) + 2-BTP(1) <=> CF3CCH(84) + S(164)
S(125) + 2-BTP(1) <=> S(125) + S(164)
S(125) + 2-BTP(1) <=> S(125) + S(164)
S(125) + 2-BTP(1) <=> S(125) + S(164)
CH2O(20) + 2-BTP(1) <=> CH2O(20) + S(164)
CH2O(20) + 2-BTP(1) <=> CH2O(20) + S(164)
CH2O(20) + 2-BTP(1) <=> CH2O(20) + S(164)
O2(4) + S(164) <=> O2(157) + 2-BTP(1)
O2(4) + S(164) <=> O2(4) + 2-BTP(1)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
CH2CO(28) + 2-BTP(1) <=> CH2CO(28) + S(164)
BR(90) + 2-BTP(1) <=> BR(90) + S(164)
BR(90) + 2-BTP(1) <=> S(144)
C2H4(30) + 2-BTP(1) <=> C2H4(30) + S(164)
O2(157) + CF3CCH(84) <=> O2(4) + CF3CCH

Trying it out

In [63]:
print(len(unique_reactions_2))
for i in range(len(unique_reactions_2)): 
    
    print(str(unique_reactions_2[i]))
    
    
###130 model (1): has 1306 reactions 
### 129 model (2): has 1262 reactions

###1306-1262 = 44, checked when i use unique_reactions_1
###when I use unique_reactions_2, i assumed I would get none since the 129 model is smaller than the 130.
  #I get 59 unique reactions in the 129 model, why?
    

0


In [57]:
# print(len(model_1_reactions))
# for i in range(len(model_1_reactions)): 
#     if str(model_1_reactions[i]) == 'CH2CHO(35) <=> CO(15) + CH3(19)':
#          print(str(model_1_reactions[i]))

print(len(common_reactions))
for i in range(len(common_reactions)):
    if str(common_reactions[i][0]) == 'CH2CHO(35) <=> CO(15) + CH3(19)':
        print(str(common_reactions[i][0]))   
print(len(model_2_reactions))
    
#these match, so good! 
    
    

1256
CH2CHO(35) <=> CO(15) + CH3(19)
CH2CHO(35) <=> CO(15) + CH3(19)
1256


'O2(4) + H(8) <=> O(9) + OH(2)'